<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/Transformer_TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [58]:
class CustomDataset(Dataset):
  def __init__(self, train=True):
    self.root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"
    self.url = self.root_url + ("FordA_TRAIN.tsv" if train else "FordA_TEST.tsv")
    self.data = np.loadtxt(self.url, delimiter="\t")
    self.x = self.data[:, 1:]
    self.y = self.data[:, 0]
  
  def __len__(self,):
    return len(self.data)
  
  def __getitem__(self, idx):
    inputs = self.x[idx]
    labels = self.y[idx]
    return (inputs, labels)

  def collate_fn(self, batch):
    inputs = []
    labels = []
    for sample in batch:
      inputs.append(sample[0])
      labels.append(sample[1])
    return {
        'inputs': torch.tensor(inputs),
        'labels': torch.tensor(labels, dtype=torch.long),
    }